# COVID-19 Dashboard
Amit Kulkarni. DM for feature requests: <a href="https://twitter.com/kulkarni_amit">@kulkarni_amit</a>

This dashboard is built using <a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data">public datasets</a> provided by Johns Hopkins University on the COVID-19 (coronavirus) pandemic.

In [42]:
from datetime import datetime
currentTime = datetime.now().astimezone().strftime('%I:%M %p (%Z) on %a %b %d, %Y.')
print('Last updated:', currentTime)

Last updated: 02:55 PM (Pacific Standard Time) on Tue Nov 10, 2020.


In [43]:
import pandas as pd
import numpy as np
import math

from urllib.request import urlopen
import json

import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [44]:
# Common Library Functions
def PercentFormatter(x):
    return "{:.0%}".format(x) if not math.isnan(x) and not math.isinf(x) else "-" 

def IntegerFormatter(x):
    return "{:,.0f}".format(x) if not math.isnan(x) and not math.isinf(x) and (x != 0) else "-" 

LogLinearScale = list([
    dict(active=0,
         type='buttons',
         direction='right',
         x=0.45,
         xanchor="left",
         y=1.1,
         yanchor="top",
         buttons=list([
            dict(label='Linear Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'yaxis': {'type': 'linear'}}]),
            dict(label='Logarithmic Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'yaxis': {'type': 'log'}}])
            ]),
        )
    ])

DEFAULT_DAYS = 270
MONTH_DAYS = 31

In [45]:
# Read all the files
dfConfirmedGlobal = pd.read_csv(r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
dfDeathsGlobal = pd.read_csv(r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

dfConfirmedUSCounties = pd.read_csv(r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
dfDeathsUSCounties = pd.read_csv(r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
dfCensusCounties = pd.read_csv(r'https://raw.githubusercontent.com/arkulkarni/COVID-19-Analysis/master/county-level-2018-population.csv', encoding = 'ISO-8859-1')

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [46]:
dfConfirmedUSStates = dfConfirmedUSCounties.groupby('Province_State').sum()
dfConfirmedUSStates = dfConfirmedUSStates[dfConfirmedUSStates.columns[5:]]
dfDeathsUSStates = dfDeathsUSCounties.groupby('Province_State').sum()
dfDeathsUSStates = dfDeathsUSStates[dfDeathsUSStates.columns[6:]]
dfCensusCounties['FIPS'] = dfCensusCounties.STATE * 1000 + dfCensusCounties.COUNTY
dfCensusCountiesPopulation = dfCensusCounties[dfCensusCounties.AGEGRP == 0]
dfConfirmedUSCounties.FIPS = dfConfirmedUSCounties.FIPS.map(lambda x : x if not math.isnan(x) else np.random.randint(-100, high=0))

# Key Facts - World and US

In [47]:
trace1 = go.Indicator(
    mode = "number+delta",
    value = dfConfirmedGlobal.sum()[-1],
    delta = {'reference': dfConfirmedGlobal.sum()[-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "Worldwide Confirmed Cases"},
    domain = {'x': [0, 0.5], 'y': [0.6, 1]})

trace2 = go.Indicator(
    mode = "number+delta",
    value = dfConfirmedUSStates.sum()[-1],
    delta = {'reference': dfConfirmedUSStates.sum()[-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "US Confirmed Cases"},
    domain = {'x': [0.5, 1], 'y': [0.6, 1]})


trace5 = go.Indicator(
    mode = "number+delta",
    value = dfDeathsGlobal.sum()[-1],
    delta = {'reference': dfDeathsGlobal.sum()[-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "Worldwide Deaths"},
    domain = {'x': [0, 0.5], 'y': [0, 0.4]})

trace6 = go.Indicator(
    mode = "number+delta",
    value = dfDeathsUSStates.sum()[-1],
    delta = {'reference': dfDeathsUSStates.sum()[-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "US Deaths"},
    domain = {'x': [0.5, 1], 'y': [0, 0.4]})

data = [trace1, trace2, trace5, trace6]
py.iplot(dict(data=data))

# Key Facts - Local

In [48]:
trace3 = go.Indicator(
    mode = "number+delta",
    value = dfConfirmedUSStates.loc['Washington'][-1],
    delta = {'reference': dfConfirmedUSStates.loc['Washington'][-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "WA Confirmed Cases"},
    domain = {'x': [0, 0.5], 'y': [0.6, 1]})

trace4 = go.Indicator(
    mode = "number+delta",
    value = dfConfirmedUSCounties[dfConfirmedUSCounties.FIPS == 53033].iloc[0][-1],
    delta = {'reference': dfConfirmedUSCounties[dfConfirmedUSCounties.FIPS == 53033].iloc[0][-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "King County Confirmed Cases"},
    domain = {'x': [0.5, 1], 'y': [0.6, 1]})

trace7 = go.Indicator(
    mode = "number+delta",
    value = dfDeathsUSStates.loc['Washington'][-1],
    delta = {'reference': dfDeathsUSStates.loc['Washington'][-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "WA Deaths"},
    domain = {'x': [0, 0.5], 'y': [0, 0.4]})

trace8 = go.Indicator(
    mode = "number+delta",
    value = dfDeathsUSCounties[dfDeathsUSCounties.FIPS == 53033].iloc[0][-1],
    delta = {'reference': dfDeathsUSCounties[dfDeathsUSCounties.FIPS == 53033].iloc[0][-2], 
             'increasing':{'color':'red'},
             'decreasing':{'color':'green'}},
    title = {'text': "King County Deaths"},
    domain = {'x': [0.5, 1], 'y': [0, 0.4]})


data = [trace3, trace4, trace7, trace8]
py.iplot(dict(data=data))

# United States - Aggregate Data

In [49]:
dfUSConfirmed = dfConfirmedGlobal[dfConfirmedGlobal['Country/Region'] == 'US'].copy()
dfUSDeaths =  dfDeathsGlobal[dfConfirmedGlobal['Country/Region'] == 'US'].copy()
dfUSConfirmed['Category'] = 'Confirmed Cases'
dfUSConfirmed = dfUSConfirmed.set_index(['Country/Region', 'Category'])
dfUSDeaths['Category'] = 'Deaths'
dfUSDeaths = dfUSDeaths.set_index(['Country/Region', 'Category'])

dfUSConfirmedDailyNew = dfUSConfirmed.diff(axis=1)
dfUSDeathsDailyNew = dfUSDeaths.diff(axis=1)

dfUSConfirmed7dayMA = dfUSConfirmedDailyNew.rolling(window=7, axis=1).mean()
dfUSDeaths7dayMA = dfUSDeathsDailyNew.rolling(window=7, axis=1).mean()

trace1 = go.Bar(
    x = dfUSConfirmedDailyNew.columns[-DEFAULT_DAYS:],
    y = dfUSConfirmedDailyNew.iloc[0][-DEFAULT_DAYS:], name = 'New Confirmed Cases')

trace2 = go.Bar(
    x = dfUSDeathsDailyNew.columns[-DEFAULT_DAYS:],
    y = dfUSDeathsDailyNew.iloc[0][-DEFAULT_DAYS:], name = 'New Deaths')

trace3 = go.Scatter(
    x = dfUSConfirmed7dayMA.columns[-DEFAULT_DAYS:],
    y = dfUSConfirmed7dayMA.iloc[0][-DEFAULT_DAYS:], name = '7-day avg Confirmed Cases')

trace4 = go.Scatter(
    x = dfUSDeaths7dayMA.columns[-DEFAULT_DAYS:],
    y = dfUSDeaths7dayMA.iloc[0][-DEFAULT_DAYS:], name = '7-day avg Deaths')

data = [trace1, trace2, trace3, trace4]
layout = dict(title = 'United States - Daily New Cases and Deaths', 
              yaxis={'side': 'right'}, 
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})

py.iplot(dict(data=data, layout=layout))
dfUSDisplay = pd.concat([dfUSConfirmedDailyNew, dfUSDeathsDailyNew])
dfUSDisplay[dfUSDisplay.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)


In [50]:
trace1 = go.Scatter(
    x = dfUSConfirmed.columns[-DEFAULT_DAYS:],
    y = dfUSConfirmed.iloc[0][-DEFAULT_DAYS:], name = 'Confirmed Cases')

trace2 = go.Scatter(
    x = dfUSDeaths.columns[-DEFAULT_DAYS:],
    y = dfUSDeaths.iloc[0][-DEFAULT_DAYS:], name = 'Deaths')

data = [trace1, trace2]
layout = dict(title = 'United States - Cumulative Cases and Deaths', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'},
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})

py.iplot(dict(data=data, layout=layout))

dfUSCombined = pd.concat([dfUSConfirmed, dfUSDeaths])
dfUSCombined[dfUSCombined.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

# US State by State Data

In [51]:
dfConfirmedUSStatesDailyNew = dfConfirmedUSStates.diff(axis=1)
dfConfirmedUSStatesDailyNew7dayMA=dfConfirmedUSStatesDailyNew.rolling(window=7, axis=1).mean()
dfConfirmedUSStatesDailyNew7dayMA = dfConfirmedUSStatesDailyNew7dayMA.sort_values(dfConfirmedUSStatesDailyNew7dayMA.columns[-1], ascending=False).head(10)

data = [{
    'x': row[1].index[-DEFAULT_DAYS:],
    'y': row[1].values[-DEFAULT_DAYS:],
    'name': row[0]
}  for row in dfConfirmedUSStatesDailyNew7dayMA.iterrows()]

layout = dict(title = 'US States Daily New Confirmed Cases - 7 Day Average', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'},
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))

dfConfirmedUSStatesDailyNew7dayMA[dfConfirmedUSStatesDailyNew7dayMA.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
Province_State,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Illinois,"2,381","2,563","2,690","2,867","2,900","3,032","3,316","3,419","3,636","3,689","3,812","4,023","4,156","4,140","4,502","4,475","4,706","4,747","5,000","5,203","5,701","5,949","6,367","6,580","6,940","7,144","7,654","8,127","8,776","9,209","9,830"
Texas,"4,070","4,033","3,827","4,035","4,149","4,259","4,471","4,647","5,083","5,120","5,066","5,178","4,841","5,011","5,703","5,429","5,908","6,070","6,122","6,753","6,554","6,271","6,401","6,286","6,505","6,854","7,253","7,539","7,543","7,731","7,961"
Georgia,"1,157","1,212","1,228","1,236","1,208","1,262","1,260","1,302","1,307","1,286","1,305","1,307","1,328","1,416","1,460","1,481","1,510","1,562","1,611","1,616","1,495","1,598","1,580","1,577","5,879","6,016","6,111","6,315","6,321","6,297","6,345"
Wisconsin,"2,395","2,510","2,548","2,727","2,840","2,928","3,053","2,661","2,279","4,421","4,622","4,802","4,785","4,897","5,565","6,104","4,112","4,229","4,195","4,412","4,514","4,676","4,644","4,714","4,800","5,104","5,267","5,417","5,662","5,784","5,922"
California,"3,270","3,211","3,201","3,301","3,325","3,268","3,168","3,029","2,929","3,105","3,205","3,105","3,602","3,935","4,296","4,415","4,399","4,449","4,620","4,326","4,176","4,066","4,183","4,282","4,359","4,521","4,744","5,026","5,317","5,262","5,841"
Florida,"2,047","2,576","2,593","2,661","2,704","2,711","2,788","3,366","2,933","2,958","3,091","2,986","3,300","3,335","3,396","3,374","3,612","3,703","3,985","3,790","4,062","3,757","4,111","4,293","4,341","4,385","4,679","4,630","4,933","5,212","5,108"
Michigan,"1,169","1,169","1,239","1,284","1,338","1,489","1,615","1,665","1,665","1,818","1,876","1,917","1,880","1,858","2,072","2,072","2,221","2,336","2,580","2,852","3,038","3,113","3,113","3,522","3,631","3,747","4,031","4,160","4,524","4,524","4,855"
Ohio,"1,356","1,406","1,459","1,475","1,563","1,654","1,698","1,824","1,863","1,921","2,002","2,049","2,084","2,137","2,226","2,333","2,372","2,443","2,477","2,644","2,833","2,842","2,984","3,097","3,343","3,552","3,748","3,914","4,290","4,467","4,724"
Minnesota,"1,149","1,206","1,234","1,262","1,327","1,312","1,441","1,466","1,506","1,571","1,565","1,521","1,578","1,495","1,576","1,570","1,562","1,715","1,836","2,023","2,229","2,336","2,410","2,607","2,794","3,068","3,222","3,549","3,783","4,313","4,453"


In [52]:
dfUSConfirmedTop = dfConfirmedUSStates.sort_values(dfConfirmedUSStates.columns[-1], ascending=False).head(10)

data = [{
    'x': row[1].index[-DEFAULT_DAYS:],
    'y': row[1].values[-DEFAULT_DAYS:],
    'name': row[0]
}  for row in dfUSConfirmedTop.iterrows()]

layout = dict(title = 'US States - Confirmed Cases', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'}, 
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))
dfUSConfirmedTop = dfUSConfirmedTop[dfUSConfirmedTop.columns[-MONTH_DAYS:]]
dfUSConfirmedTop['percent_of_total'] = dfConfirmedUSStates[dfConfirmedUSStates.columns[-1]]/sum(dfConfirmedUSStates[dfConfirmedUSStates.columns[-1]])

colFormats = dict.fromkeys(list(a for a in dfUSConfirmedTop.columns[1:]), IntegerFormatter)
colFormats['percent_of_total'] = PercentFormatter
dfUSConfirmedTop.style.format(colFormats)

,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,percent_of_total
Province_State,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Texas,815915,"818,426","821,110","826,816","832,736","837,566","843,487","848,442","854,006","856,948","862,277","868,981","871,453","878,567","888,362","892,006","898,302","904,769","911,835","918,721","924,447","932,261","936,816","942,303","950,302","959,811","969,490","977,222","985,061","990,930","998,027",10%
California,852410,"855,190","857,913","861,310","865,204","868,148","871,253","873,614","875,692","879,645","883,746","886,939","893,364","898,799","903,684","906,596","910,438","914,888","919,276","923,648","928,028","932,143","935,878","940,411","945,401","950,920","956,854","963,211","969,362","972,713","981,297",10%
Florida,728921,"734,491","736,024","738,749","741,632","744,988","748,437","752,481","755,020","756,727","760,389","762,533","768,091","771,780","776,251","778,636","782,013","786,311","790,426","794,624","800,216","802,547","807,412","812,063","816,700","821,123","827,380","832,625","837,077","843,897","847,821",8%
New York,473143,"474,286","475,315","476,708","477,940","479,400","481,107","482,891","484,281","485,279","486,480","488,506","490,134","491,771","493,832","495,464","496,655","498,646","500,677","503,176","505,431","507,543","509,735","511,368","513,689","515,815","518,812","522,021","525,608","529,036","532,180",5%
Illinois,319451,"322,178","324,918","327,771","330,637","334,651","339,757","343,386","347,631","350,744","354,457","358,799","363,740","368,740","374,901","378,958","383,687","387,687","393,797","400,159","408,648","416,547","423,527","429,749","436,265","443,803","453,738","465,540","477,978","487,987","498,560",5%
Georgia,330269,"331,409","332,311","333,304","334,601","336,241","337,850","339,384","340,558","341,310","342,438","343,750","345,535","347,759","349,605","350,923","351,881","353,372","355,025","356,848","358,225","360,790","361,982","362,921","394,526","397,137","399,625","402,427","405,037","406,061","407,333",4%
North Carolina,229752,"231,471","232,747","234,481","236,407","238,939","241,623","243,725","246,028","247,172","248,750","250,592","252,992","255,708","258,292","260,099","261,742","263,883","266,136","269,021","271,830","274,635","276,692","278,028","280,377","282,802","285,661","288,569","291,245","293,339","294,860",3%
Tennessee,212649,"214,717","217,682","218,829","220,538","222,827","223,493","226,139","228,744","232,061","233,569","235,861","237,907","241,513","244,087","247,587","249,866","251,774","254,220","256,880","259,488","260,672","261,426","264,587","266,357","269,802","271,771","273,144","278,215","281,851","287,770",3%
Wisconsin,147560,"150,236","152,192","155,471","158,578","162,325","166,186","166,186","166,186","183,142","187,828","192,191","195,823","200,466","205,139","208,917","211,928","217,429","221,559","226,704","232,061","237,869","241,422","244,927","251,031","257,287","263,571","269,982","277,503","281,910","286,380",3%


In [53]:
dfUSDeathsTop = dfDeathsUSStates.sort_values(dfDeathsUSStates.columns[-1], ascending=False).head(10)

data = [{
    'x': row[1].index[-DEFAULT_DAYS:],
    'y': row[1].values[-DEFAULT_DAYS:],
    'name': row[0]
}  for row in dfUSDeathsTop.iterrows()]

updatemenus = list([
    dict(active=0,
         type='buttons',
         direction='right',
         x=0.45,
         xanchor="left",
         y=1.1,
         yanchor="top",
         buttons=list([
            dict(label='Linear Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'yaxis': {'type': 'linear'}}]),
            dict(label='Logarithmic Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'yaxis': {'type': 'log'}}])
            ]),
        )
    ])

layout = dict(title = 'US Deaths', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'}, 
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))
dfUSDeathsTop = dfUSDeathsTop[dfUSDeathsTop.columns[-MONTH_DAYS:]]
dfUSDeathsTop['percent_of_total'] = dfDeathsUSStates[dfDeathsUSStates.columns[-1]]/sum(dfDeathsUSStates[dfDeathsUSStates.columns[-1]])

colFormats = dict.fromkeys(list(a for a in dfUSDeathsTop.columns[1:]), IntegerFormatter)
colFormats['percent_of_total'] = PercentFormatter
dfUSDeathsTop.style.format({'percent_of_total': PercentFormatter})

,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,percent_of_total
Province_State,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
New York,33466,33469,33482,33490,33500,33524,33533,33547,33559,33575,33582,33589,33614,33625,33634,33642,33650,33666,33675,33694,33724,33730,33752,33763,33773,33793,33857,33872,33884,33903,33705,14%
Texas,16943,16971,16995,17078,17188,17283,17375,17440,17465,17481,17558,17652,17659,17773,17915,17955,17978,18061,18162,18276,18385,18465,18502,18534,18632,18771,18907,19046,19146,19184,19223,8%
California,16567,16579,16586,16644,16729,16832,16910,16956,16979,16982,17055,17167,17256,17305,17343,17358,17386,17460,17529,17559,17618,17659,17671,17689,17745,17808,17860,17931,17961,17973,18021,8%
Florida,15186,15364,15412,15531,15595,15736,15830,15917,15967,16021,16105,16210,16267,16340,16417,16429,16449,16505,16571,16648,16720,16761,16789,16834,16890,16922,16961,17014,17100,17121,17179,7%
New Jersey,16171,16174,16175,16182,16191,16197,16202,16204,16211,16214,16227,16245,16263,16273,16281,16285,16292,16306,16324,16332,16339,16350,16354,16357,16371,16391,16403,16416,16425,16429,16440,7%
Illinois,9221,9230,9243,9273,9320,9373,9425,9452,9474,9496,9537,9605,9647,9688,9751,9775,9792,9838,9889,9945,9994,10040,10075,10093,10161,10216,10313,10397,10488,10538,10563,4%
Massachusetts,9587,9604,9617,9630,9647,9672,9702,9723,9737,9753,9758,9780,9810,9830,9839,9864,9881,9888,9924,9951,9975,9991,10013,10023,10035,10062,10085,10106,10129,10149,10163,4%
Pennsylvania,8324,8330,8347,8361,8391,8413,8434,8442,8465,8475,8516,8546,8574,8608,8636,8647,8659,8683,8707,8743,8774,8800,8803,8812,8842,8878,8923,8964,9001,9005,9008,4%
Georgia,7393,7416,7429,7454,7470,7492,7556,7607,7638,7657,7674,7704,7729,7766,7808,7809,7827,7844,7876,7923,7955,7979,7981,7999,8029,8072,8126,8156,8193,8194,8223,3%


# Washington State - Aggregate Data

In [54]:
dfWAConfirmed = dfConfirmedUSStates[dfConfirmedUSStates.index == 'Washington'].copy()
dfWADeaths =  dfDeathsUSStates[dfDeathsUSStates.index == 'Washington'].copy()
dfWAConfirmed['Category'] = 'Confirmed Cases'
dfWAConfirmed = dfWAConfirmed.reset_index().set_index(['Province_State', 'Category'])
dfWADeaths['Category'] = 'Deaths'
dfWADeaths = dfWADeaths.reset_index().set_index(['Province_State', 'Category'])

dfWAConfirmedDailyNew = dfWAConfirmed.diff(axis=1)
dfWADeathsDailyNew = dfWADeaths.diff(axis=1)

dfWAConfirmed7dayMA = dfWAConfirmedDailyNew.rolling(window=7, axis=1).mean()
dfWADeaths7dayMA = dfWADeathsDailyNew.rolling(window=7, axis=1).mean()

trace1 = go.Bar(
    x = dfWAConfirmedDailyNew.columns[-DEFAULT_DAYS:],
    y = dfWAConfirmedDailyNew.iloc[0][-DEFAULT_DAYS:], name = 'New Confirmed Cases')

trace2 = go.Bar(
    x = dfWADeathsDailyNew.columns[-DEFAULT_DAYS:],
    y = dfWADeathsDailyNew.iloc[0][-DEFAULT_DAYS:], name = 'New Deaths')

trace3 = go.Scatter(
    x = dfWAConfirmed7dayMA.columns[-DEFAULT_DAYS:],
    y = dfWAConfirmed7dayMA.iloc[0][-DEFAULT_DAYS:], name = '7-day avg Confirmed Cases')

trace4 = go.Scatter(
    x = dfWADeaths7dayMA.columns[-DEFAULT_DAYS:],
    y = dfWADeaths7dayMA.iloc[0][-DEFAULT_DAYS:], name = '7-day avg Deaths')

data = [trace1, trace2, trace3, trace4]
layout = dict(title = 'WA state - Daily New Cases and Deaths', 
              yaxis={'side': 'right'},
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})

py.iplot(dict(data=data, layout=layout))

dfWADisplay = pd.concat([dfWAConfirmedDailyNew, dfWADeathsDailyNew])
dfWADisplay[dfWADisplay.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

In [55]:
trace1 = go.Scatter(
    x = dfWAConfirmed.columns[-DEFAULT_DAYS:],
    y = dfWAConfirmed.iloc[0][-DEFAULT_DAYS:], name = 'Confirmed Cases')

trace2 = go.Scatter(
    x = dfWADeaths.columns[-DEFAULT_DAYS:],
    y = dfWADeaths.iloc[0][-DEFAULT_DAYS:], name = 'Deaths')

data = [trace1, trace2]
layout = dict(title = 'WA state - Cumulative Cases and Deaths', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'}, 
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})

py.iplot(dict(data=data, layout=layout))

dfWACombined = pd.concat([dfWAConfirmed, dfWADeaths])
dfWACombined[dfWACombined.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

# Washington State County by County Data

In [56]:
dfWAByCountyConfirmed = dfConfirmedUSCounties[dfConfirmedUSCounties.Province_State == 'Washington']
dfWAByCountyConfirmed = dfWAByCountyConfirmed.set_index('Admin2')

dfWAByCountyConfirmed = dfWAByCountyConfirmed.sort_values(dfWAByCountyConfirmed.columns[-1], ascending = False)
dfWAByCountyConfirmedTop = dfWAByCountyConfirmed.head(10)
dfWAByCountyConfirmedTop = dfWAByCountyConfirmedTop[dfWAByCountyConfirmedTop.columns[-DEFAULT_DAYS:]]


data = [{
    'x': row[1].index,
    'y': row[1].values,
    'name': row[0]
}  for row in dfWAByCountyConfirmedTop.iterrows()]



layout = dict(title = 'WA Counties - Cumulative Confirmed Cases', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'},
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))

## Daily new cases by county

In [57]:
dfWAByCountyConfirmedTopNew = dfWAByCountyConfirmedTop.diff(axis=1).fillna(0).head(5)
dfWAByCountyConfirmedTopNew.style.format(IntegerFormatter)

,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
Admin2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
King,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,5,3,5,7,10,20,7,13,12,-,33,74,80,58,59,-,101,81,-7,131,100,141,106,130,-,189,218,-,500,82,2,169,-,326,131,111,269,164,155,202,198,231,145,164,-,123,71,77,205,-,272,-,119,86,153,105,102,124,82,56,102,104,67,77,98,96,76,83,105,54,84,60,61,69,75,78,35,101,46,43,50,37,28,39,-,114,43,9,28,60,26,44,99,31,54,87,54,53,46,40,39,33,-,21,61,83,28,63,65,35,78,98,93,57,42,113,87,120,99,137,86,124,158,127,231,149,98,159,164,77,102,135,149,-,257,252,136,112,233,208,187,200,166,131,177,208,191,223,168,162,138,139,169,-,365,111,102,123,139,213,144,217,131,137,101,133,177,190,183,132,122,163,146,243,37,212,116,135,40,139,179,167,127,104,90,55,99,95,130,144,71,89,-,87,106,121,134,89,49,48,89,99,116,124,65,63,43,91,110,106,129,108,95,39,130,162,220,167,106,93,103,133,196,165,93,-,-,492,169,144,159,216,108,126,110,211,172,184,268,165,190,80,190,232,332,382,216,272,114,589,390,516,531,491,332
Yakima,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2,2,-,-,1,2,-,3,3,3,-,9,22,14,5,16,58,7,14,-,63,16,49,37,37,17,20,33,14,64,24,3,76,21,39,33,52,45,31,33,18,-7,44,53,39,19,12,36,46,75,69,24,76,17,60,98,32,107,95,83,52,30,56,50,165,36,117,74,74,33,43,-,218,146,50,40,183,138,89,127,164,68,121,93,110,225,143,156,169,-,154,141,136,119,66,147,54,126,138,214,154,43,109,55,117,152,69,92,88,63,57,142,79,47,93,145,62,57,115,66,-,188,203,32,79,369,134,101,85,60,76,51,55,94,96,72,71,58,46,61,-,96,56,30,34,61,25,43,59,56,42,22,23,55,56,67,35,20,20,15,40,38,38,20,14,13,10,16,30,29,14,6,15,12,19,13,27,25,12,-,25,16,17,12,12,18,15,8,19,16,31,23,-,18,15,15,16,31,17,10,8,18,11,29

# World Wide Data

In [58]:
chinaDataConfirmed = dfConfirmedGlobal[dfConfirmedGlobal['Country/Region'] == 'China'].sum()
chinaDataConfirmed['Province/State', 'Country/Region'] = [np.NaN, 'China']

dfConfirmedWorldTop = dfConfirmedGlobal[dfConfirmedGlobal['Country/Region'] != 'China'].sort_values(dfConfirmedGlobal.columns[-1], ascending=False).head(10)
dfConfirmedWorldTop = dfConfirmedWorldTop.append(chinaDataConfirmed, ignore_index=True)
dfConfirmedWorldTop = dfConfirmedWorldTop.sort_values(dfConfirmedGlobal.columns[-1], ascending=False)
dfConfirmedWorldTop = dfConfirmedWorldTop.set_index('Country/Region')

data = [{
    'x': row[1].index[-DEFAULT_DAYS:],
    'y': row[1].values[-DEFAULT_DAYS:],
    'name': row[0]
}  for row in dfConfirmedWorldTop.iterrows()]

layout = dict(title = 'World Cumulative Confirmed Cases', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'},
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))


dfConfirmedWorldTop[dfConfirmedWorldTop.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
US,"7,694,782","7,740,674","7,781,236","7,834,257","7,892,627","7,956,848","8,025,110","8,082,183","8,131,522","8,198,105","8,259,044","8,321,033","8,393,691","8,476,223","8,560,099","8,622,046","8,688,041","8,763,634","8,841,246","8,931,449","9,029,237","9,118,074","9,194,530","9,276,410","9,399,466","9,501,771","9,627,471","9,752,360","9,880,828","9,990,608","10,110,552"
India,"7,053,806","7,120,538","7,175,880","7,239,389","7,307,097","7,370,468","7,432,680","7,494,551","7,550,273","7,597,063","7,651,107","7,706,946","7,761,312","7,814,682","7,864,811","7,909,959","7,946,429","7,990,322","8,040,203","8,088,851","8,137,119","8,184,082","8,229,313","8,267,623","8,313,876","8,364,086","8,411,724","8,462,080","8,507,754","8,553,657","8,591,730"
Brazil,"5,082,637","5,094,979","5,103,408","5,113,628","5,140,863","5,169,386","5,200,300","5,224,362","5,224,362","5,250,727","5,273,954","5,298,772","5,323,630","5,353,656","5,380,635","5,394,128","5,409,854","5,439,641","5,468,270","5,494,376","5,516,658","5,535,605","5,545,705","5,554,206","5,566,049","5,590,025","5,590,025","5,631,181","5,653,561","5,664,115","5,675,032"
France,"702,148","702,148","745,104","766,421","788,117","818,707","843,475","843,471","843,471","918,679","939,147","965,451","1,007,026","1,048,842","1,048,842","1,048,842","1,172,754","1,206,014","1,240,862","1,288,478","1,337,693","1,373,036","1,419,326","1,471,091","1,507,078","1,547,831","1,605,171","1,665,403","1,665,403","1,790,817","1,810,653"
Russia,"1,278,245","1,291,687","1,305,093","1,318,783","1,332,824","1,346,380","1,361,317","1,376,020","1,390,824","1,406,667","1,422,775","1,438,219","1,453,923","1,471,000","1,487,260","1,503,652","1,520,800","1,537,142","1,553,028","1,570,446","1,588,433","1,606,267","1,624,648","1,642,665","1,661,096","1,680,579","1,699,695","1,720,063","1,740,172","1,760,420","1,781,997"
Spain,"861,112","861,112","888,968","896,086","908,056","921,374","936,560","936,560","936,560","974,449","988,322","1,005,295","1,026,281","1,046,132","1,046,132","1,046,132","1,098,320","1,116,738","1,136,503","1,160,083","1,185,678","1,185,678","1,185,678","1,240,697","1,259,366","1,284,408","1,306,316","1,328,832","1,328,832","1,328,832","1,381,218"
Argentina,"883,882","894,206","903,730","917,035","931,967","949,063","965,609","979,119","989,680","1,002,662","1,018,999","1,037,325","1,053,650","1,069,368","1,081,336","1,090,589","1,102,301","1,116,609","1,130,533","1,143,800","1,157,179","1,166,924","1,173,533","1,183,131","1,195,276","1,205,928","1,217,028","1,228,814","1,236,851","1,242,182","1,250,499"
United Kingdom,"590,844","603,716","617,688","634,920","654,644","673,622","689,257","705,428","722,409","741,212","762,542","789,229","810,467","830,998","854,010","873,800","894,690","917,575","942,275","965,340","989,745","1,011,660","1,034,914","1,053,864","1,073,882","1,099,059","1,123,197","1,146,484","1,171,441","1,192,013","1,213,363"
Colombia,"902,747","911,316","919,084","924,098","930,159","936,982","945,354","952,371","959,572","965,883","974,139","981,700","990,373","998,942","1,007,711","1,015,885","1,025,052","1,033,218","1,041,935","1,053,122","1,063,151","1,074,184","1,083,321","1,093,256","1,099,392","1,108,086","1,117,977","1,127,733","1,136,447","1,143,887","1,149,064"


In [59]:
dfConfirmedWorldTop5 = dfConfirmedWorldTop.head(5)

dfConfirmedWorldTop5DailyDataCols = dfConfirmedWorldTop5[dfConfirmedWorldTop5.columns[4:]]
dfConfirmedWorldTop5Daily = pd.concat([dfConfirmedWorldTop5[dfConfirmedWorldTop5.columns[:4]], dfConfirmedWorldTop5DailyDataCols.diff(axis=1)], axis=1)

dfConfirmedWorldTop5Daily7dayMA = dfConfirmedWorldTop5Daily.rolling(window=7, axis=1).mean()

data = [{
    'x': row[1].index[-DEFAULT_DAYS:],
    'y': row[1].values[-DEFAULT_DAYS:],
    'name': row[0]
}  for row in dfConfirmedWorldTop5Daily7dayMA.iterrows()]

layout = dict(title = 'World Daily New Confirmed Cases - 7 Day Average', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'}, 
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))

dfConfirmedWorldTop5Daily7dayMA[dfConfirmedWorldTop5Daily7dayMA.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
US,"47,830","49,197","49,471","50,978","52,201","53,206","54,927","55,343","55,835","59,553","60,684","61,201","62,406","64,445","68,274","70,075","69,991","72,084","74,316","76,823","79,002","79,711","81,783","84,053","90,833","94,361","99,432","103,303","108,965","113,725","119,163"
India,"72,062","70,960","70,114","68,894","67,349","66,331","64,751","62,964","61,391","60,169","58,817","57,121","55,835","54,572","52,894","51,384","49,909","48,459","47,608","46,791","46,062","45,610","45,622","45,885","46,222","46,269","46,125","46,423","46,239","46,335","46,301"
Brazil,"25,115","25,670","25,168","20,641","20,024","20,135","20,630","20,246","18,483","21,046","22,904","22,558","22,035","21,908","22,325","24,252","22,732","23,670","24,214","24,392","23,286","22,139","21,654","20,622","18,058","17,394","13,664","16,360","16,851","16,916","17,261"
France,"14,311","14,311","15,558","17,127","17,706","19,502","20,190","20,189","20,189","24,796","24,675","25,333","26,903","29,338","29,339","29,339","36,296","38,124","39,344","40,207","41,264","46,313","52,926","42,620","43,009","43,853","45,242","46,816","41,767","53,070","48,509"
Russia,"11,369","11,807","12,185","12,501","12,938","13,254","13,678","13,968","14,162","14,511","14,856","15,056","15,363","15,669","15,891","16,118","16,305","16,338","16,401","16,646","16,776","17,001","17,285","17,409","17,708","18,222","18,464","18,804","19,129","19,396","19,905"


In [60]:
chinaDataDeaths = dfDeathsGlobal[dfDeathsGlobal['Country/Region'] == 'China'].sum()
chinaDataDeaths['Province/State', 'Country/Region'] = [np.NaN, 'China']

dfDeathsWorldTop = dfDeathsGlobal[dfDeathsGlobal['Country/Region'] != 'China'].sort_values(dfDeathsGlobal.columns[-1], ascending=False).head(10)
dfDeathsWorldTop = dfDeathsWorldTop.append(chinaDataDeaths, ignore_index=True)
dfDeathsWorldTop = dfDeathsWorldTop.sort_values(dfDeathsGlobal.columns[-1], ascending=False)
dfDeathsWorldTop = dfDeathsWorldTop.set_index('Country/Region')


data = [{
    'x': row[1].index[-DEFAULT_DAYS:],
    'y': row[1].values[-DEFAULT_DAYS:],
    'name': row[0]
}  for row in dfDeathsWorldTop.iterrows()]

layout = dict(title = 'World Deaths', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'},
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))
dfDeathsWorldTop[dfDeathsWorldTop.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
US,"214,546","214,946","215,269","216,074","217,059","217,882","218,774","219,489","219,879","220,331","221,271","222,398","223,254","224,190","225,113","225,457","225,940","226,933","227,929","228,910","229,908","230,817","231,216","231,714","232,850","233,954","235,127","236,281","237,317","237,779","238,256"
Brazil,"150,198","150,488","150,689","150,998","151,747","152,460","153,214","153,675","153,675","154,176","154,837","155,403","155,900","156,471","156,903","157,134","157,397","157,946","158,456","158,969","159,477","159,884","160,074","160,253","160,496","161,106","161,106","162,015","162,269","162,397","162,628"
India,"108,334","109,150","109,856","110,586","111,266","112,161","112,998","114,031","114,610","115,197","115,914","116,616","117,306","117,956","118,534","119,014","119,502","120,010","120,527","121,090","121,641","122,111","122,607","123,097","123,611","124,315","124,985","125,562","126,121","126,611","127,059"
Mexico,"83,642","83,781","83,945","84,420","84,898","85,285","85,704","86,059","86,167","86,338","86,893","87,415","87,415","88,312","88,743","88,924","89,171","89,814","90,309","90,773","91,289","91,753","91,895","92,100","92,593","93,228","93,772","94,323","94,808","95,027","95,027"
United Kingdom,"42,760","42,825","42,875","43,018","43,155","43,293","43,429","43,579","43,646","43,726","43,967","44,158","44,347","44,571","44,745","44,896","44,998","45,365","45,675","45,955","46,229","46,555","46,717","46,853","47,250","47,742","48,120","48,475","48,888","49,044","49,238"
Italy,"36,140","36,166","36,205","36,246","36,289","36,372","36,427","36,474","36,543","36,616","36,705","36,832","36,968","37,059","37,210","37,338","37,479","37,700","37,905","38,122","38,321","38,618","38,826","39,059","39,412","39,764","40,192","40,638","41,063","41,394","41,750"
France,"32,356","32,356","32,456","32,714","32,788","32,876","33,054","33,054","33,054","33,374","33,636","33,777","33,939","34,236","34,236","34,236","34,746","35,268","35,493","35,728","36,273","36,494","36,725","37,141","37,564","38,370","38,730","39,557","39,861","40,131","40,678"
Spain,"32,929","32,929","33,124","33,204","33,413","33,553","33,775","33,775","33,775","33,992","34,210","34,366","34,521","34,752","34,752","34,752","35,031","35,298","35,466","35,639","35,878","35,878","35,878","36,257","36,495","38,118","38,486","38,833","38,833","38,833","39,345"
Iran,"28,293","28,544","28,816","29,070","29,349","29,605","29,870","30,123","30,375","30,712","31,034","31,346","31,650","31,985","32,320","32,616","32,953","33,299","33,714","34,113","34,478","34,864","35,298","35,738","36,160","36,579","36,985","37,409","37,832","38,291","38,749"


In [61]:
dfDeathsWorldTop5 = dfDeathsWorldTop.head(5)

dfDeathsWorldTop5DailyDataCols = dfDeathsWorldTop5[dfDeathsWorldTop5.columns[4:]]
dfDeathsWorldTop5Daily = pd.concat([dfDeathsWorldTop5[dfDeathsWorldTop5.columns[:4]], dfDeathsWorldTop5DailyDataCols.diff(axis=1)], axis=1)

dfDeathsWorldTop5Daily7dayMA = dfDeathsWorldTop5Daily.rolling(window=7, axis=1).mean()

data = [{
    'x': row[1].index[-DEFAULT_DAYS:],
    'y': row[1].values[-DEFAULT_DAYS:],
    'name': row[0]
}  for row in dfDeathsWorldTop5Daily7dayMA.iterrows()]

layout = dict(title = 'World Daily New Deaths - 7 Day Average', 
              updatemenus=LogLinearScale, 
              yaxis={'side': 'right'}, 
              legend= {'x': 0.001, 'xanchor': 'left','y': 0.99})
py.iplot(dict(data=data, layout=layout))

dfDeathsWorldTop5Daily7dayMA[dfDeathsWorldTop5Daily7dayMA.columns[-MONTH_DAYS:]].style.format(IntegerFormatter)

,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
US,712,720,700,714,723,702,693,706,705,723,742,763,767,774,803,797,801,809,790,808,817,815,823,825,845,861,888,910,929,938,935
Brazil,602,591,573,501,503,500,511,497,455,498,548,522,491,465,461,494,460,444,436,438,429,426,420,408,364,379,305,363,341,332,339
India,936,924,898,862,820,810,797,814,780,763,761,764,735,708,643,629,615,585,559,541,526,511,513,514,514,541,556,560,573,572,566
Mexico,680,670,295,296,310,313,315,345,341,342,353,360,304,373,383,394,405,417,413,480,425,430,424,418,397,417,428,433,436,447,418
United Kingdom,63,68,72,82,91,100,107,117,117,122,136,143,151,163,167,179,182,200,217,230,237,259,260,265,269,295,309,321,333,332,341


In [62]:
# Detailed US County By County Analysis

In [63]:
# dfConfirmedUSCountiesLatest = dfConfirmedUSCounties[['FIPS', dfConfirmedUSCounties.columns[-1]]]
# dfDeathsUSCountiesLatest = dfDeathsUSCounties[['FIPS', dfDeathsUSCounties.columns[-1]]]

# dfLatestWithPopulation = dfConfirmedUSCountiesLatest.merge(dfDeathsUSCountiesLatest, how='left', on='FIPS')
# dfLatestWithPopulation.columns = ['FIPS', 'confirmed_cases', 'deaths']
# dfLatestWithPopulation = dfLatestWithPopulation.merge(dfCensusCountiesPopulation, how='left', on='FIPS')
# dfLatestWithPopulation = dfLatestWithPopulation[~np.isnan(dfLatestWithPopulation.COUNTY)]
# dfLatestWithPopulation['confirmed_cases_per_thousand_population'] = 1000 * dfLatestWithPopulation.confirmed_cases/dfLatestWithPopulation.TOT_POP
# dfLatestWithPopulation['deaths_per_thousand_population'] = 1000 * dfLatestWithPopulation.deaths/dfLatestWithPopulation.TOT_POP

# topUSCountiesPerCapita = dfLatestWithPopulation[dfLatestWithPopulation.confirmed_cases > 50].sort_values('confirmed_cases_per_thousand_population', ascending=False).head(30)

# dfTopUSCounties = dfConfirmedUSCountiesLatest[dfConfirmedUSCountiesLatest[dfConfirmedUSCountiesLatest.columns[-1]] > 100]
# # dfLatestWithPopulation[dfLatestWithPopulation.confirmed_cases > 100]
# # topUSCountiesPerCapita

# trace1 = go.Choropleth(
#     locations=dfTopUSCounties['FIPS'], 
#     z = dfConfirmedUSCountiesLatest[dfConfirmedUSCountiesLatest.columns[-1]],
#     geojson=counties,
#     colorscale = 'Plasma',
#     colorbar = dict(tickmode='array', tickvals = [1, 100, 200,  500, 1000, 10000, 50000, 1000000]),
#     colorbar_title = 'Confirmed Cases',
# )

# data = [trace1]
# layout = dict(title = 'Top US Counties', geo_scope='usa')
# py.iplot(dict(data=data, layout=layout))
# dfTopUSCounties.describe()

# topUSCountiesPerCapita